# 4.9 Part One - Table of Contents

-Import libraries and data 

-Wrangle the new customers dataframe

-Quality and consistency checks on the new customers dataframe

-Merge customer data with the full merged file

-Export merged file

## Steps 2-3 import libraries and customer dataframe

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Remember project folder file path
path = r'C:\Users\rstev\Documents\03_22 Instacart Basket Analysis'

In [3]:
# Import customer dataframe
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [4]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [6]:
df_cust.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

## Step 4. Wrangle data

In [7]:
# Change id column to string
df_cust['user_id'] = df_cust['user_id'].astype('str')

In [8]:
df_cust['user_id'].dtype

dtype('O')

In [9]:
# Renaming columns
df_cust.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)

In [10]:
df_cust.rename(columns = {'STATE' : 'State'}, inplace = True)

In [11]:
df_cust.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)

In [12]:
df_cust.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

In [13]:
df_cust.head()

,user_id,First Name,Last Name,Gender,State,Age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## Step 5. Quality and consistency checks

In [14]:
# Check for mixed data types
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First Name


In [15]:
# Converting all in first name col to string
df_cust['First Name'] = df_cust['First Name'].astype('str')

In [16]:
df_cust['First Name'].dtype

dtype('O')

In [17]:
# Check for missing data (no missing)
df_cust.isnull().sum()

user_id                 0
First Name              0
Last Name               0
Gender                  0
State                   0
Age                     0
date_joined             0
number_of_dependants    0
marital_status          0
income                  0
dtype: int64

In [18]:
# Look for full dupes, i.e., rows with every column the same (no dupes)
df_dups = df_cust[df_cust.duplicated()]

In [19]:
df_dups

,user_id,First Name,Last Name,Gender,State,Age,date_joined,number_of_dependants,marital_status,income


In [20]:
# All quality checks look good. Reviewed summary stats (mean, min, max, count) above and all looked good.

## Step 6. Combine customer data with the rest of the Instacart data

In [21]:
# Import merged dataframe
df_ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_4.pkl'))

In [22]:
# Creating a smaller subset because not enough memory to run on full data frame
df_ords_prods_small = df_ords_prods.iloc[:10000000]

In [23]:
# Check shape of both data frames
df_ords_prods_small.shape

(10000000, 24)

In [24]:
df_cust.shape

(206209, 10)

In [25]:
df_ords_prods_small.head()

,order_id,user_id,order_number,orders_day_of_week,hour_order_placed,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_day,Busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_spend,spend_flag,median_order_frequency,order_freq_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Least busy days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [26]:
df_ords_prods_small.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'hour_order_placed', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', '_merge', 'price_range_loc', 'busiest_day',
       'Busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'average_spend', 'spend_flag', 'median_order_frequency',
       'order_freq_flag'],
      dtype='object')

In [27]:
# Delete existing merge flag from file
df_ords_prods_small = df_ords_prods_small.drop(columns = ['_merge'])

In [28]:
# Change user_id type to string in ords_prods
df_ords_prods_small['user_id'] = df_ords_prods_small['user_id'].astype('str')

In [29]:
# Merge in customer data
ords_prods_cust_merged = df_ords_prods_small.merge(df_cust, on = ['user_id'], indicator = True)

In [30]:
ords_prods_cust_merged.head()

,order_id,user_id,order_number,orders_day_of_week,hour_order_placed,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First Name,Last Name,Gender,State,Age,date_joined,number_of_dependants,marital_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [31]:
ords_prods_cust_merged.shape

(10000000, 33)

In [32]:
# Confirm merge using merge flag
ords_prods_cust_merged['_merge'].value_counts()

both          10000000
left_only            0
right_only           0
Name: _merge, dtype: int64

In [34]:
# Export as pickle file
ords_prods_cust_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_merged_small.pkl'))